In [3]:
import numpy as np
import cv2
import os

In [4]:
os.chdir('C:/Digital Engineering/openCV/Project_Automatic_Face_Blur')

In [5]:
# Load image

img=cv2.imread('./images/happy_parents.jpg')

cv2.imshow('orignal',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# Load face detection model
face_detection_model= cv2.dnn.readNetFromCaffe('./model/deploy.prototxt.txt','./model/res10_300x300_ssd_iter_140000_fp16.caffemodel')

In [24]:
def face_blur(img):
    
    # blob from image
    image=img.copy()
    #mask image
    face_mask=np.zeros(image.shape[:2],dtype='uint8')

    blob=cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=True)
    face_detection_model.setInput(blob)
    detections=face_detection_model.forward()

    h,w=image.shape[:2]
    for i in range (detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>0.5:
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            box=box.astype(int)
            cv2.rectangle(face_mask,(box[0],box[1]),(box[2],box[3]),255,-1)


    inverse_face_mask= cv2.bitwise_not(face_mask)

    # Apply mask image to orignal image
    bg_img=cv2.bitwise_and(image,image,mask=face_mask)
    fg_img=cv2.bitwise_and(image,image,mask=inverse_face_mask)

    # blur background image with Gaussian Blur
    bg_img_blur=cv2.GaussianBlur(bg_img,(51,51),0)

    # addition
    face_blur=cv2.add(fg_img,bg_img_blur)
    
    return face_blur


In [25]:
detections.shape

(1, 1, 200, 7)

In [ ]:
cv2.imshow('face_mask',face_mask)
cv2.imshow('inverse_face_mask',inverse_face_mask)
cv2.imshow('backgroung',bg_img)
cv2.imshow('foreground',fg_img)
cv2.imshow('backgroung_blur',bg_img_blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:

face_blur_img=face_blur(img)
cv2.imshow('face_blur',face_blur_img)
cv2.waitKey(0)
cv2.destroyAllWindows()